# Introdução
## Contexto do projeto
Todos os anos letivos, a equipa administrativa do **IPCA** enfrenta dificuldades na criação dos horários das aulas, devido às restrições complexas relacionadas com **professores, cursos, horários e salas disponíveis**. Este é um problema típico de **Satisfação de Restrições (*Constraint Satisfaction Problem* - CSP)**, onde é necessário encontrar soluções que satisfaçam um conjunto de *hard constraints* e *soft constraints*.

## Objetivo do projeto
O principal objetivo do projeto é criar um sistema que produza horários **válidos, otimizados e que respeitem todas as restrições**.

## Equipa de desenvolvimento
- Duarte Pereira — Nº 27959
- Hugo Especial — Nº 27963
- Paulo Gonçalves — Nº 27966 
- Marco Cardoso — Nº 27969 
- Hugo Pereira — Nº 27970

---

# *Design* do agente

## Formulação do problema como um Problema de Satisfação de Restrições (CSP)
Formular um problema de CSP exige identificar claramente três elementos:
- Variáveis, domínios e restrições.

E o objetivo é encontrar uma atribuição de valores que satisfaça todas as restrições.

## Definição de variáveis, domínio e restrições
### **Variáveis**:
$X = \{ Sala, Dia, Hora \}$

### **Domínio**:  
- $D(Sala) = \{ (Room A, Room C, Room G, Lab02) \}$
- $D(Dia) = \{ (Segunda, Terça, Quarta, Quinta, Sexta) \}$
- $D(Hora) = \{ (9h00 - 11h00, 11h00 - 13h00, 14h00 - 16h00, 16h00 - 18h00) \}$

### **Restrições**:
Para facilitar a formulação do problema, as restrições foram divididas em categorias.

#### Restrições temporais:
- Cada aula dura **2 horas**.
```python
def check_duration(professor, hora_inicio, disciplina) -> bool:
```

- Todas as turmas têm **10 aulas semanais**.
``` python
def check_weekly_classes(turma, disciplina) -> bool:
```

- Cada curso pode ter **1 ou 2 aulas por semana**.

```python
def check_weekly_course_classes(curso, disciplina) -> bool:
```

- Uma turma **não pode ter mais de 3 aulas por dia**.
```python
def check_daily_classes(turma, dia) -> bool:
```

- **Aulas online (máx. 3)** devem ser realizadas **no mesmo dia**.
```python
def check_online_classes(disciplina, dia) -> bool:
```

- Aulas da mesma unidade curricular devem ocorrer em **dias distintos**.
```python
def check_distinct_day_classes(disciplina, dias) -> bool:
```

- Cada turma deve ter, se possível, **apenas 4 dias de aulas por semana**.
```python
def check_weekly_days(turma, dias) -> bool:
```

- As aulas de cada dia devem ser **consecutivas**.
``` python
def check_consecutive_classes(dia, disciplinas) -> bool:
```

#### Restrições de professores:
- O horário deve respeitar a **disponibilidade dos professores**.
```python
def check_professor_availability(professor, dia, hora) -> bool:
```

#### Restrições de salas:
- Algumas aulas são obrigatoriamente atribuídas a **salas específicas**.
``` python
def check_specific_class(disciplina, sala) -> bool:
```

- O número de **salas diferentes por turma** deve ser minimizado.
```python
def check_different_classes(turma, salas) -> bool:
```

### Importação dos dados
Os dados foram importados a partir de ficheiros CSV (*Comma Separated Value*), utilizando Python. Cada ficheiro contém informações sobre diferentes entidades (cursos, professores, salas, turmas, unidades curriculares e disponibilidades.)

In [ ]:
import csv

def load_csv(path):

    data = []
    with open(path, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data.append(row)
    return data


def load_courses(path='dados/cursos.csv'):
    cursos = load_csv(path)
    for c in cursos:
        c['id'] = int(c['id'])
        c['n_turmas'] = int(c['n_turmas'])
    return cursos


def load_teachers(path='dados/professores.csv'):
    professores = load_csv(path)
    for p in professores:
        p['id'] = int(p['id'])
    return professores


def load_rooms(path='dados/salas.csv'):
    salas = load_csv(path)
    for s in salas:
        s['id'] = int(s['id'])
    return salas


def load_curricular_units(path='dados/unidades_curriculares.csv'):
    ucs = load_csv(path)
    for u in ucs:
        u['id'] = int(u['id'])
        u['curso_id'] = int(u['curso_id'])
        u['n_aulas_semana'] = int(u['n_aulas_semana'])
    return ucs


def load_availabilities(path='dados/disponibilidades.csv'):
    disp = load_csv(path)
    for d in disp:
        d['prof_id'] = int(d['prof_id'])
        d['hora_inicio'] = int(d['hora_inicio'])
        d['hora_fim'] = int(d['hora_fim'])
    return disp


def load_classes(path='dados/turmas.csv'):
    turmas = load_csv(path)
    for t in turmas:
        t['id'] = int(t['id'])
        t['curso_id'] = int(t['curso_id'])
        t['n_alunos'] = int(t['n_alunos'])
    return turmas


def load_all():

    return {
        'cursos': load_courses(),
        'professores': load_teachers(),
        'salas': load_rooms(),
        'unidades_curriculares': load_curricular_units(),
        'turmas': load_classes(),
        'disponibilidades': load_availabilities()
    }

dados = load_all()
print("=== Cursos ===")
for curso in dados['cursos']:
    print(curso)

print("\n=== Professores ===")
for prof in dados['professores']:
    print(prof)

print("\n=== Salas ===")
for sala in dados['salas']:
    print(sala)
    
print("\n=== Turmas ===")
for turma in dados['turmas']:
    print(turma)

print("\n=== Unidades Curriculares ===")
for uc in dados['unidades_curriculares']:
    print(uc)

print("\n=== Disponibilidades ===")
for disp in dados['disponibilidades']:
    print(disp)

## Função heurística para implementar as *soft constraints*


O processo de geração de horários é dividido em duas fases.
Na primeira, **o *solver* da biblioteca *constraint* gera todas as soluções válidas que respeitam as *hard constraints***, garantindo **apenas a validade das restrições obrigatórias**.

Na segunda, **as soluções são avaliadas e ordenadas através de uma função heurística**, que calcula uma **pontuação com base nas *soft constraints***, refletindo o grau de otimização das preferências.

A função heurística calcula uma pontuação para cada solução, considerando critérios como:
- o número de dias distintos que uma UC ocupa.
- se uma turma tiver 4 ou menos dias de aulas.
- se as aulas de um dia forem consecutivas.
- o menor número de salas diferentes.

A pontuação total da solução é a **soma dos valores obtidos em cada uma destas preferências**.

As **soluções com maior pontuação são consideradas ótimas**, uma vez que cumprem melhor as preferências definidas.

---
